# Dublês de teste

Normalmente quando falamos de mocks, estamos nos referindo a um componente simulado de software. Porém existem vários tipos de simulações que podem ser feitas que podem ajudar a escrever os testes.

Vamos conceituar cada um dos tipos de dublês de teste e como poderiam ser utilizados para testar a função apresentada acima.

> 💡 As definições em inglês foram retiradas de: https://martinfowler.com/bliki/TestDouble.html

## Dummy

> Dummy objects are passed around but never actually used. Usually they are just used to fill parameter lists.

São objetos “dummy”, ou seja falsos, fictícios, que serão utilizados apenas para preencher a lista de parâmetros obrigatórios, mas não serão utilizados.

In [31]:
// FormatToISO8601 formats a date to the ISO 8601 standard.
// Example: 
//   date := time.Date(2024, time.December, 9, 15, 4, 5, 0, time.UTC)
//   fmt.Println(FormatToISO8601(date)) // Output: "2024-12-09T15:04:05Z"
func FormatToISO8601(date time.Time) string {
	return date.Format(time.RFC3339)
}

In [32]:
func TestFormatToISO8601WithDummyDate(t *testing.T) {
	dummyDate := time.Date(2024, time.December, 9, 15, 4, 5, 0, time.UTC)
	formattedDate := FormatToISO8601(dummyDate)
	expected := "2024-12-09T15:04:05Z"
	
	if formattedDate != expected {
		t.Errorf("expected '%s', got '%s'", expected, formattedDate)
	}
}

%test

=== RUN   TestFormatToISO8601WithDummyDate
--- PASS: TestFormatToISO8601WithDummyDate (0.00s)
PASS


## Fake

> Fake objects actually have working implementations, but usually take some shortcut which makes them not suitable for production (an InMemoryTestDatabase is a good example).

São objetos falsos, com implementações concretas, porém simplificadas. Um bom exemplo são objetos que representam bancos de dados ou arquivos, porém com implementações em memória.

In [33]:
type User struct {
	Name  string
	Email string
}
 
type UserStore interface {
	Save(ctx context.Context, user User) error
}

In [34]:
type FakeUserStore struct {
	users map[string]User
}

func (f *FakeUserStore) Save(ctx context.Context, user User) error {
	select {
	case <-ctx.Done():
		return ctx.Err()
	default:
		f.users[user.Name] = user
		return nil
	}
}

func NewFakeUserStore() *FakeUserStore {
	return &FakeUserStore{
		users: make(map[string]User),
	}
}

In [35]:
func RegisterUser(store UserStore, user User) error {
	err := store.Save(context.Background(), user)
	if err != nil {
		return fmt.Errorf("error saving user: %v", err)
	}
	return nil
}

func TestRegisterUser(t *testing.T) {
	t.Run("successfully registering a user", func(t *testing.T) {
		userStore := NewFakeUserStore()
		user := User{Name: "John Doe", Email: "john.doe@example.com"}
		err := RegisterUser(userStore, user)
		if err != nil {
			t.Fatalf("expected no error, but got: %v", err)
		}
		if _, exists := userStore.users[user.Name]; !exists {
			t.Errorf("expected user '%s' to be saved, but not found", user.Name)
		}
	})

	t.Run("failure registering a user with cancelled context", func(t *testing.T) {
		userStore := NewFakeUserStore()
		user := User{Name: "Jane Doe", Email: "jane.doe@example.com"}
		ctx, cancel := context.WithCancel(context.Background())
		cancel()
		err := userStore.Save(ctx, user)
		if err == nil {
			t.Fatalf("expected error due to context cancellation, but no error occurred")
		}
	})
}

%test

=== RUN   TestRegisterUser
=== RUN   TestRegisterUser/successfully_registering_a_user
=== RUN   TestRegisterUser/failure_registering_a_user_with_cancelled_context
--- PASS: TestRegisterUser (0.00s)
    --- PASS: TestRegisterUser/successfully_registering_a_user (0.00s)
    --- PASS: TestRegisterUser/failure_registering_a_user_with_cancelled_context (0.00s)
PASS


## Stub

> Stubs provide canned answers to calls made during the test, usually not responding at all to anything outside what’s programmed in for the test.

São substitutos que fornecem respostas previamente definidas, simulando assim o comportamento esperado.

In [36]:
type ForGettingTaxRates interface {
	TaxRate(amount float64) float64
}

type TaxService struct {
	api ForGettingTaxRates
}

func (s *TaxService) CalculateTax(amount float64) float64 {
	taxRate := s.api.TaxRate(amount)
	return amount * taxRate
}


In [37]:
type StubTaxAPI struct{}

func (s *StubTaxAPI) TaxRate(amount float64) float64 {
	if amount > 0 && amount <= 100 {
		return 0.07
	}
	if amount > 100 {
		return 0.15
	}
	return 0.0
}

In [38]:
func TestCalculateTax(t *testing.T) {
	stubAPI := &StubTaxAPI{}
	taxService := &TaxService{api: stubAPI}

	t.Run("should calculate tax for amounts less than or equal to 100", func(t *testing.T) {
		amount := 50.0
		expectedTax := 3.5  // 50 * 0.07 (7% de imposto)

		tax := taxService.CalculateTax(amount)
		assert.InDelta(t, expectedTax, tax, 0.01)
	})

	t.Run("should calculate tax for amounts greater than 100", func(t *testing.T) {
		amount := 150.0
		expectedTax := 22.5  // 150 * 0.15 (15% de imposto)

		tax := taxService.CalculateTax(amount)
		assert.InDelta(t, expectedTax, tax, 0.01)
	})

	t.Run("should return tax 0 for negative or zero amounts", func(t *testing.T) {
		amount := -10.0

		tax := taxService.CalculateTax(amount)
		assert.Equal(t, 0.0, tax)
	})
}

%test

=== RUN   TestCalculateTax
=== RUN   TestCalculateTax/should_calculate_tax_for_amounts_less_than_or_equal_to_100
=== RUN   TestCalculateTax/should_calculate_tax_for_amounts_greater_than_100
=== RUN   TestCalculateTax/should_return_tax_0_for_negative_or_zero_amounts
--- PASS: TestCalculateTax (0.00s)
    --- PASS: TestCalculateTax/should_calculate_tax_for_amounts_less_than_or_equal_to_100 (0.00s)
    --- PASS: TestCalculateTax/should_calculate_tax_for_amounts_greater_than_100 (0.00s)
    --- PASS: TestCalculateTax/should_return_tax_0_for_negative_or_zero_amounts (0.00s)
PASS


## Spies 

> Spies are stubs that also record some information based on how they were called. One form of this might be an email service that records how many messages it was sent.

São stubs mas “espionam” como são invocados e mantém isto como informação a ser utilizada nas asserções.

In [39]:
type SpyForGettingTaxRates struct {
	callCount int
	calls     []float64
	returnValue float64
}

func (s *SpyForGettingTaxRates) TaxRate(amount float64) float64 {
	s.callCount++
	s.calls = append(s.calls, amount)
	return s.returnValue // Return the mocked tax rate value
}

func (s *SpyForGettingTaxRates) GetCallCount() int {
	return s.callCount
}

func (s *SpyForGettingTaxRates) GetCalls() []float64 {
	return s.calls
}

In [40]:
func TestCalculateTaxWithSpy_TaxRateCalled(t *testing.T) {
	tests := []struct {
		name          string
		amount        float64
		expectedTax   float64
		expectedValue float64
	}{
		{"Tax for 100", 100, 0.1, 10},
		{"Tax for 200", 200, 0.1, 20},
		{"Tax for 150", 150, 0.2, 30},
	}

	
	for _, tt := range tests {
		t.Run(tt.name, func(t *testing.T) {
			spy := &SpyForGettingTaxRates{returnValue: tt.expectedTax}
			taxService := &TaxService{api: spy}
            
			taxAmount := taxService.CalculateTax(tt.amount)
            
			if spy.GetCallCount() != 1 {
				t.Errorf("Esperava que TaxRate fosse chamado 1 vez, mas foi chamado %d vezes", spy.GetCallCount())
			}
            
			if spy.GetCalls()[0] != tt.amount {
				t.Errorf("Esperava que TaxRate fosse chamado com %f, mas foi chamado com %f", tt.amount, spy.GetCalls()[0])
			}

			if taxAmount != tt.expectedValue {
				t.Errorf("Esperava que o imposto fosse %f, mas obteve %f", tt.expectedValue, taxAmount)
			}
		})
	}
}

## Mocks

> Mocks are pre-programmed with expectations which form a specification of the calls they are expected to receive. They can throw an exception if they receive a call they don’t expect and are checked during verification to ensure they got all the calls they were expecting.

A preocupação de mocks é verificar se o comportamento do dublê foi o esperado, fazendo asserções se o mock foi invocado, se os parâmetros na invocação foram corretos e o número de vezes em que foi invocado.

ℹ️ Foi utilizado a biblioteca https://github.com/stretchr/testify/mock para este exemplo.

In [41]:
type EmailService interface {
	SendEmail(to string, subject string, body string) bool
}


type MockEmailService struct {
	mock.Mock
}

func (m *MockEmailService) SendEmail(to, subject, body string) bool {
	args := m.Called(to, subject, body)
	return args.Bool(0)
}

type NotificationService struct {
	emailService EmailService
}

func (n *NotificationService) Notify(to, subject, body string) bool {
	return n.emailService.SendEmail(to, subject, body)
}

In [42]:
func TestNotificationService_Notify(t *testing.T) {
	mockEmailService := new(MockEmailService)
	mockEmailService.On("SendEmail", "test@example.com", "Test Subject", "Test Body").Return(true)

	notificationService := &NotificationService{
		emailService: mockEmailService,
	}

	result := notificationService.Notify("test@example.com", "Test Subject", "Test Body")

	assert.True(t, result)

	mockEmailService.AssertExpectations(t)
}

%test

=== RUN   TestNotificationService_Notify
--- PASS: TestNotificationService_Notify (0.00s)
PASS


🤖 Para automação da criação de mocks para suas interfaces as bilbiotecas [mockery](https://vektra.github.io/mockery/latest/) e [uber-go/mock](https://github.com/uber-go/mock) são as mais utilizadas.

## 📚 Materiais interessantes:
1. https://quii.gitbook.io/learn-go-with-tests
2. https://blog.jetbrains.com/go/2022/11/22/comprehensive-guide-to-testing-in-go/
3. https://gobyexample.com/testing